In [ ]:
from datasets import Dataset
items = [
    {'messages': [{'role': 'user', 'content': 'Hello!'}]},
]

ds = Dataset.from_list(items)
def f(batch):
    messages_list = batch['messages']
    for messages in messages_list:
        for msg in messages:
            if msg['role'] == 'user':
                msg['content'] += ' world 1'
    return batch
ds = ds.map(f, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [43]:
ds[0]

{'messages': [{'content': 'Hello! world', 'role': 'user'}]}